<a href="https://colab.research.google.com/github/JuneHou/BirdSpeciesCNN/blob/main/DataCollection_Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import models
from torch.utils.data import random_split
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Appendix: Data Collection

- Setup path of train, test and valid file

In [ ]:
train_dir = '/content/drive/MyDrive/NEU/DataMining/Project/train'
test_dir = '/content/drive/MyDrive/NEU/DataMining/Project/test'
val_dir = '/content/drive/MyDrive/NEU/DataMining/Project/valid'

- All images has dimension in 224 X 224 X 3, in which the 3 represents RGB images. The input dimension 224 x 224 is same as the model required. Data is normalized while transform to pytorch datasets.

In [ ]:
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(root=train_dir, transform=transformations)
test_data = datasets.ImageFolder(root=test_dir, transform=transformations)
valid_data = datasets.ImageFolder(root=val_dir, transform=transformations)

- The class_dict file shows the images and the classes it belongs to. The classes will used in the test model.

In [ ]:
class_dict = pd.read_csv('/content/drive/MyDrive/NEU/DataMining/Project/class_dict.csv')
classes = list(class_dict['class'])
print(len(classes))

350


In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 16

In [ ]:
# obtain training indices that will be used for validation
num_train, num_valid = len(train_data), len(valid_data)
train_index, valid_index = list(range(num_train)), list(range(num_valid))
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
valid_sampler = SubsetRandomSampler(valid_index)
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size,
                                           sampler = train_sampler, num_workers = num_workers)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size,
                                          sampler = valid_sampler, num_workers = num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size,
                                         num_workers = num_workers)

> Train Test Setup

In [ ]:
def train (n_epoch, base_model, train_loader, optimizer, criterion, valid_loader):
  for epoch in range(n_epoch):  # loop over the dataset multiple times

    print('Epoch: ', epoch+1)
    class_correct = list(0. for i in range(350))
    class_total = list(0. for i in range(350))
    
    train_loss = 0
    valid_loss = 0
    
    base_model.train()
    for images, labels in train_loader:
        
        images = images.to(device)
        
        #Reset Grads
        optimizer.zero_grad()
        
        #Forward ->
        outputs = base_model(images).to("cpu")
        
        #Calculate Loss & Backward, Update Weights (Step)
        loss = criterion(outputs, labels)
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    
    
    #Validation
    base_model.eval()
    for images, labels in valid_loader:
        
        images = images.to(device)
        
        #Forward ->
        preds = base_model(images).to("cpu")
        
        #Calculate Loss
        loss = criterion(preds, labels)
        valid_loss += loss.item() * images.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(preds, 1)

        # compare predictions to true label
        correct = np.squeeze(pred.eq(labels.data.view_as(pred)))

        # calculate test accuracy for each object class
        for i in range(len(labels)):
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    print('\nValid Accuracy: %2d%%' % 
          (100. * np.sum(class_correct) / np.sum(class_total)))
    
    
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    
    #Print Epoch Statistics
    print("Train Loss = {}".format(round(train_loss, 4)))
    print("Valid Loss = {}".format(round(valid_loss, 4)))

In [ ]:
def test (classes, base_model, test_loader, criterion):
  # initialize lists to monitor test loss and accuracy
  test_loss = 0.0
  class_correct = list(0. for i in range(len(classes)))
  class_total = list(0. for i in range(len(classes)))
  base_model.eval() # prep model for evaluation

  for data, target in test_loader:

      data = data.to(device)

      # forward pass: compute predicted outputs by passing inputs to the model
      output = base_model(data).to("cpu")
      # calculate the loss
      loss = criterion(output, target)
      # update test loss 
      test_loss += loss.item()*data.size(0)

      # convert output probabilities to predicted class
      _, pred = torch.max(output, 1)

      # compare predictions to true label
      correct = np.squeeze(pred.eq(target.data.view_as(pred)))

      # calculate test accuracy for each object class
      for i in range(len(target)):
          label = target.data[i]
          class_correct[label] += correct[i].item()
          class_total[label] += 1
        
  for i in range(len(classes)):
      if class_total[i] > 0:
          print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
      else:
          print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

  # calculate and print avg test loss
  test_loss = test_loss/len(test_loader.sampler)
  print('Test Loss: {:.6f}\n'.format(test_loss))
  print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))

> Model Setup

- The weight will not been updated in all layers except the fc layer. The fc layer as the output layer has the input dimension same as the pre-trained model, because both use 224 x 224 image, and the output dimension is the length of the classes (350).
- To improve the speed, model is trained in the GPU.
- The Criterion is Cross Entropy
- The optimizer is Adam optimizer with learning rate equals 0.001.
- Both model trained in 10 epoches.

# Benchmark Model -- VGG 19

VGG network is build based on the AlexNet, which published in 2012, and the name is the abbreviation of the group named as Visual Geometry Group at Oxford’s. VGG replaced the 11x11 and 5x5 filters with a stack of 3x3 filters layer. This is based on the finding of ZfNet, which was a frontline network of 2013-ILSVRC competition, suggested that small size filters can improve the performance of the CNNs. Two most famous models are: VGG-16, with 16 CNN layers and 3 fully connected layers; VGG-19, with 19 CNN layers and 3 fully connected layers.

In [ ]:
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = models.vgg19(pretrained=True)

# turn training false for all layers, other than fc layer
for param in base_model.parameters():
    param.requires_grad = False
    
base_model.classifier

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

- Modify the 2nd fully connected layer from (4096, 4096) to (4096, 2048)
- Modify the output layer with dimension same as number of classes (350)

In [ ]:
base_model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(True),
    nn.Dropout(0.5),
    nn.Linear(4096, 2048),
    nn.ReLU(True),
    nn.Dropout(0.5),
    nn.Linear(2048, len(classes))
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr = 0.001)

- The architecture of the neural network is shown below

In [ ]:
summary(base_model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
train(10, base_model, train_loader, optimizer, criterion, valid_loader)

Epoch:  1

Valid Accuracy:  2%
Train Loss = 5.8404
Valid Loss = 5.4765
Epoch:  2

Valid Accuracy:  2%
Train Loss = 5.6864
Valid Loss = 5.39
Epoch:  3

Valid Accuracy:  2%
Train Loss = 5.6858
Valid Loss = 5.382
Epoch:  4

Valid Accuracy:  2%
Train Loss = 5.6977
Valid Loss = 5.4755
Epoch:  5

Valid Accuracy:  1%
Train Loss = 5.7436
Valid Loss = 5.5864
Epoch:  6

Valid Accuracy:  1%
Train Loss = 5.7343
Valid Loss = 5.5368
Epoch:  7

Valid Accuracy:  1%
Train Loss = 5.7518
Valid Loss = 5.6044
Epoch:  8

Valid Accuracy:  1%
Train Loss = 5.7673
Valid Loss = 5.7051
Epoch:  9

Valid Accuracy:  1%
Train Loss = 5.7722
Valid Loss = 5.6213
Epoch:  10

Valid Accuracy:  1%
Train Loss = 5.7893
Valid Loss = 5.7797


In [ ]:
test(classes, base_model, test_loader, criterion)

Test Accuracy of ABBOTTS BABBLER:  0% ( 0/ 5)
Test Accuracy of ABBOTTS BOOBY:  0% ( 0/ 5)
Test Accuracy of ABYSSINIAN GROUND HORNBILL:  0% ( 0/ 5)
Test Accuracy of AFRICAN CROWNED CRANE:  0% ( 0/ 5)
Test Accuracy of AFRICAN EMERALD CUCKOO:  0% ( 0/ 5)
Test Accuracy of AFRICAN FIREFINCH:  0% ( 0/ 5)
Test Accuracy of AFRICAN OYSTER CATCHER:  0% ( 0/ 5)
Test Accuracy of ALBATROSS:  0% ( 0/ 5)
Test Accuracy of ALBERTS TOWHEE:  0% ( 0/ 5)
Test Accuracy of ALEXANDRINE PARAKEET:  0% ( 0/ 5)
Test Accuracy of ALPINE CHOUGH:  0% ( 0/ 5)
Test Accuracy of ALTAMIRA YELLOWTHROAT:  0% ( 0/ 5)
Test Accuracy of AMERICAN AVOCET:  0% ( 0/ 5)
Test Accuracy of AMERICAN BITTERN:  0% ( 0/ 5)
Test Accuracy of AMERICAN COOT:  0% ( 0/ 5)
Test Accuracy of AMERICAN GOLDFINCH:  0% ( 0/ 5)
Test Accuracy of AMERICAN KESTREL:  0% ( 0/ 5)
Test Accuracy of AMERICAN PIPIT:  0% ( 0/ 5)
Test Accuracy of AMERICAN REDSTART:  0% ( 0/ 5)
Test Accuracy of AMETHYST WOODSTAR:  0% ( 0/ 5)
Test Accuracy of ANDEAN GOOSE:  0% ( 0/ 5

> Benchmark Model -- ResNet 18

ResNet is a network architecture build in 2016, with maximum 152 CNN layers. In ResNet, the output from the previous layer, called residual, is added to the output of the current layer. This changing decreases the error rate for deeper networks, but adds complexity of the model. Besides, ResNet gained a 28% improvement on the famous image recognition benchmark dataset named COCO (Lin et al. 2014).

The number of layers varies from 18 to 152. The ResNet 18 network contains 18 CNN layers. 

In [ ]:
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = models.resnet18(pretrained=True)

# turn training false for all layers, other than fc layer
for param in base_model.parameters():
    param.requires_grad = False
    
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Linear(num_ftrs, len(classes))
base_model = base_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr = 0.001)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

- The neural network architecture of ResNet 18 model is shown below. 

In [ ]:
summary(base_model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [ ]:
train(10, base_model, train_loader, optimizer, criterion, valid_loader)

Epoch:  1

Valid Accuracy: 88%
Train Loss = 1.7502
Valid Loss = 0.4494
Epoch:  2

Valid Accuracy: 89%
Train Loss = 0.6778
Valid Loss = 0.351
Epoch:  3

Valid Accuracy: 91%
Train Loss = 0.5307
Valid Loss = 0.2878
Epoch:  4

Valid Accuracy: 90%
Train Loss = 0.4466
Valid Loss = 0.2856
Epoch:  5

Valid Accuracy: 92%
Train Loss = 0.3987
Valid Loss = 0.2555
Epoch:  6

Valid Accuracy: 92%
Train Loss = 0.3601
Valid Loss = 0.2523
Epoch:  7

Valid Accuracy: 92%
Train Loss = 0.3277
Valid Loss = 0.2511
Epoch:  8

Valid Accuracy: 92%
Train Loss = 0.3093
Valid Loss = 0.2701
Epoch:  9

Valid Accuracy: 92%
Train Loss = 0.2845
Valid Loss = 0.2742
Epoch:  10

Valid Accuracy: 92%
Train Loss = 0.2702
Valid Loss = 0.2606


In [ ]:
test(classes, base_model, test_loader, criterion)

Test Accuracy of ABBOTTS BABBLER: 20% ( 1/ 5)
Test Accuracy of ABBOTTS BOOBY: 40% ( 2/ 5)
Test Accuracy of ABYSSINIAN GROUND HORNBILL: 100% ( 5/ 5)
Test Accuracy of AFRICAN CROWNED CRANE: 100% ( 5/ 5)
Test Accuracy of AFRICAN EMERALD CUCKOO: 60% ( 3/ 5)
Test Accuracy of AFRICAN FIREFINCH: 80% ( 4/ 5)
Test Accuracy of AFRICAN OYSTER CATCHER: 100% ( 5/ 5)
Test Accuracy of ALBATROSS: 100% ( 5/ 5)
Test Accuracy of ALBERTS TOWHEE: 100% ( 5/ 5)
Test Accuracy of ALEXANDRINE PARAKEET: 80% ( 4/ 5)
Test Accuracy of ALPINE CHOUGH: 100% ( 5/ 5)
Test Accuracy of ALTAMIRA YELLOWTHROAT: 60% ( 3/ 5)
Test Accuracy of AMERICAN AVOCET: 100% ( 5/ 5)
Test Accuracy of AMERICAN BITTERN: 100% ( 5/ 5)
Test Accuracy of AMERICAN COOT: 100% ( 5/ 5)
Test Accuracy of AMERICAN GOLDFINCH: 100% ( 5/ 5)
Test Accuracy of AMERICAN KESTREL: 100% ( 5/ 5)
Test Accuracy of AMERICAN PIPIT: 80% ( 4/ 5)
Test Accuracy of AMERICAN REDSTART: 60% ( 3/ 5)
Test Accuracy of AMETHYST WOODSTAR: 80% ( 4/ 5)
Test Accuracy of ANDEAN GOOSE:

> Benchmark Model -- ResNet 50

Resnet-50 has 50 CNN layer, which use of 3-layer bottleneck blocks

In [ ]:
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = models.resnet50(pretrained=True)

# turn training false for all layers, other than fc layer
for param in base_model.parameters():
    param.requires_grad = False
    
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Linear(num_ftrs, len(classes))
base_model = base_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr = 0.001)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
summary(base_model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
train(10, base_model, train_loader, optimizer, criterion, valid_loader)

Epoch:  1

Valid Accuracy: 84%
Train Loss = 1.6601
Valid Loss = 0.5384
Epoch:  2

Valid Accuracy: 88%
Train Loss = 0.7783
Valid Loss = 0.4271
Epoch:  3

Valid Accuracy: 89%
Train Loss = 0.6524
Valid Loss = 0.4019
Epoch:  4

Valid Accuracy: 90%
Train Loss = 0.5597
Valid Loss = 0.4331
Epoch:  5

Valid Accuracy: 90%
Train Loss = 0.5296
Valid Loss = 0.4303
Epoch:  6

Valid Accuracy: 91%
Train Loss = 0.474
Valid Loss = 0.3661
Epoch:  7

Valid Accuracy: 90%
Train Loss = 0.4497
Valid Loss = 0.4456
Epoch:  8

Valid Accuracy: 90%
Train Loss = 0.4178
Valid Loss = 0.4561
Epoch:  9

Valid Accuracy: 92%
Train Loss = 0.3992
Valid Loss = 0.3603
Epoch:  10

Valid Accuracy: 91%
Train Loss = 0.3818
Valid Loss = 0.4493


In [ ]:
test(classes, base_model, test_loader, criterion)

Test Accuracy of ABBOTTS BABBLER: 60% ( 3/ 5)
Test Accuracy of ABBOTTS BOOBY:  0% ( 0/ 5)
Test Accuracy of ABYSSINIAN GROUND HORNBILL: 100% ( 5/ 5)
Test Accuracy of AFRICAN CROWNED CRANE: 100% ( 5/ 5)
Test Accuracy of AFRICAN EMERALD CUCKOO: 100% ( 5/ 5)
Test Accuracy of AFRICAN FIREFINCH: 80% ( 4/ 5)
Test Accuracy of AFRICAN OYSTER CATCHER: 100% ( 5/ 5)
Test Accuracy of ALBATROSS: 100% ( 5/ 5)
Test Accuracy of ALBERTS TOWHEE: 100% ( 5/ 5)
Test Accuracy of ALEXANDRINE PARAKEET: 60% ( 3/ 5)
Test Accuracy of ALPINE CHOUGH: 60% ( 3/ 5)
Test Accuracy of ALTAMIRA YELLOWTHROAT: 60% ( 3/ 5)
Test Accuracy of AMERICAN AVOCET: 100% ( 5/ 5)
Test Accuracy of AMERICAN BITTERN: 100% ( 5/ 5)
Test Accuracy of AMERICAN COOT: 100% ( 5/ 5)
Test Accuracy of AMERICAN GOLDFINCH: 100% ( 5/ 5)
Test Accuracy of AMERICAN KESTREL: 100% ( 5/ 5)
Test Accuracy of AMERICAN PIPIT: 80% ( 4/ 5)
Test Accuracy of AMERICAN REDSTART: 80% ( 4/ 5)
Test Accuracy of AMETHYST WOODSTAR: 100% ( 5/ 5)
Test Accuracy of ANDEAN GOOSE

> ResNet 101

In [ ]:
base_model = models.resnet101(pretrained=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# turn training false for all layers, other than fc layer
for param in base_model.parameters():
    param.requires_grad = False
    
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Linear(num_ftrs, len(classes))
base_model = base_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_model.parameters(), lr = 0.001)
summary(base_model, (3,224,224))

In [ ]:
train(10, base_model, train_loader, optimizer, criterion, valid_loader)

Epoch:  1

Valid Accuracy: 86%
Train Loss = 1.5594
Valid Loss = 0.477
Epoch:  2

Valid Accuracy: 88%
Train Loss = 0.7273
Valid Loss = 0.4242
Epoch:  3

Valid Accuracy: 89%
Train Loss = 0.6246
Valid Loss = 0.3865
Epoch:  4

Valid Accuracy: 89%
Train Loss = 0.5418
Valid Loss = 0.4654
Epoch:  5

Valid Accuracy: 90%
Train Loss = 0.4949
Valid Loss = 0.3587
Epoch:  6

Valid Accuracy: 90%
Train Loss = 0.4509
Valid Loss = 0.4039
Epoch:  7

Valid Accuracy: 92%
Train Loss = 0.4315
Valid Loss = 0.3334
Epoch:  8

Valid Accuracy: 91%
Train Loss = 0.411
Valid Loss = 0.4314
Epoch:  9

Valid Accuracy: 91%
Train Loss = 0.3714
Valid Loss = 0.4015
Epoch:  10

Valid Accuracy: 93%
Train Loss = 0.3608
Valid Loss = 0.3345


In [ ]:
test(classes, base_model, test_loader, criterion)

Test Accuracy of ABBOTTS BABBLER: 100% ( 5/ 5)
Test Accuracy of ABBOTTS BOOBY: 60% ( 3/ 5)
Test Accuracy of ABYSSINIAN GROUND HORNBILL: 100% ( 5/ 5)
Test Accuracy of AFRICAN CROWNED CRANE: 100% ( 5/ 5)
Test Accuracy of AFRICAN EMERALD CUCKOO: 80% ( 4/ 5)
Test Accuracy of AFRICAN FIREFINCH: 80% ( 4/ 5)
Test Accuracy of AFRICAN OYSTER CATCHER: 100% ( 5/ 5)
Test Accuracy of ALBATROSS: 100% ( 5/ 5)
Test Accuracy of ALBERTS TOWHEE: 100% ( 5/ 5)
Test Accuracy of ALEXANDRINE PARAKEET: 100% ( 5/ 5)
Test Accuracy of ALPINE CHOUGH: 100% ( 5/ 5)
Test Accuracy of ALTAMIRA YELLOWTHROAT: 20% ( 1/ 5)
Test Accuracy of AMERICAN AVOCET: 100% ( 5/ 5)
Test Accuracy of AMERICAN BITTERN: 100% ( 5/ 5)
Test Accuracy of AMERICAN COOT: 100% ( 5/ 5)
Test Accuracy of AMERICAN GOLDFINCH: 100% ( 5/ 5)
Test Accuracy of AMERICAN KESTREL: 100% ( 5/ 5)
Test Accuracy of AMERICAN PIPIT: 80% ( 4/ 5)
Test Accuracy of AMERICAN REDSTART: 100% ( 5/ 5)
Test Accuracy of AMETHYST WOODSTAR: 80% ( 4/ 5)
Test Accuracy of ANDEAN GOO

> ResNet 152

# Conclusion

With same train, valid and test datasets, and same loss function and criterion, VGG-19 model gives only 1% accuracy, but ResNet-18 gives 93% accuracy. Therefore, the structure of ResNet fits this bird species datasets better. Further exploration of ResNet50 gives 91% accuracy.

The model tuning will based comparison of difference between VGG model and ResNet model. Besides, all three pre-trained models shows overfitting issue, with valid loss increasing. Data augmentation may applied in later experiments.